In [2]:
import pandas as pd
import os
import numpy as np

In [123]:
PATH = "./wandb_classification_cnn_bp_0505.csv"
dataset = pd.read_csv(PATH)
PATH = "./wandb_classification_cnn_ccl_0505.csv"
dataset2 = pd.read_csv(PATH)
dataset = pd.concat([dataset, dataset2])

# del dataset["Created"], dataset["Runtime"], dataset["GradC"], dataset["act_B"], dataset["act_F"], dataset["epochs"], dataset["loss_scale_ssl"]
# del dataset["User"], dataset["Group"], dataset["Job Type"], dataset["Tags"], dataset["Sweep"], dataset["Description"], dataset["GPU Count"], dataset["best_epoch"], dataset["num_chn"], dataset["tmax"], dataset["warmup"], dataset["eta_min"]
# if one of the above colum is in the dataset, delete it
for col_name in dataset.columns:
    if col_name in ["Created", "Runtime", "GradC", "act_B", "act_F", "epochs", "loss_scale_ssl", "User", "Group", "Job Type", "Tags", "Sweep", "Description", "GPU Count", "best_epoch", "num_chn", "tmax", "warmup", "eta_min"]:
        del dataset[col_name]

print("Size of dataset: ", len(dataset))
dataset = dataset[dataset["State"]=="finished"]
dataset = dataset[dataset["architecture"]=="cnn_pool"]
# dataset = dataset[dataset["batchsize"]==64]
# dataset = dataset[dataset["fw_bn"]==0]
print("Size of dataset: ", len(dataset))

Size of dataset:  1328
Size of dataset:  1328


In [124]:
dataset[dataset.method=="CCL"].dataset.unique()

array(['CIFAR10', 'STL10_cls', 'CIFAR100'], dtype=object)

In [173]:
for task in ["CIFAR10", "CIFAR100", "STL10_cls"]:
    for method in ["BP", "CCL"]:
        ds = dataset[dataset["dataset"] == task]
        ds = ds[ds["method"] == method]
        if task == "CIFAR10" and method == "CCL":
            ds = ds[ds["lr_F"] != 1.2]
        ds_mean = ds.groupby(["grad_clip_F", "lr_F", "lr_B", "mmt_F", "mmt_B", "wd_F", "wd_B", "wd_B", "bw_bn", "loss_scale_C"]).mean()
        ds_count = ds.groupby(["grad_clip_F", "lr_F", "lr_B", "mmt_F", "mmt_B", "wd_F", "wd_B", "wd_B", "bw_bn", "loss_scale_C"]).count()
        ds_std = ds.groupby(["grad_clip_F", "lr_F", "lr_B", "mmt_F", "mmt_B", "wd_F", "wd_B", "wd_B", "bw_bn", "loss_scale_C"]).std()
        ds_mean = ds_mean[ds_count["test_acc"] > 2]
        ds_std = ds_std[ds_count["test_acc"] > 2]
        try:
            row_with_highest_val_acc = ds_mean["val_acc"].idxmax()
        except:
            continue
        print("dataset: ", task, " method: ", method)
        print(f"mean: {ds_mean.loc[row_with_highest_val_acc]['test_acc']*100:.2f} std: {ds_std.loc[row_with_highest_val_acc]['test_acc']*100:.2f} count: {ds_count.loc[row_with_highest_val_acc]['test_acc']}")
        # if task == "CIFAR10" and method == "CCL":
        #     assert 0 == 1
        if task == "STL10_cls" and method == "CCL":
            assert 0 == 1

dataset:  CIFAR10  method:  BP
mean: 87.12 std: 1.76 count: 10
dataset:  CIFAR10  method:  CCL
mean: 82.94 std: 0.53 count: 5
dataset:  CIFAR100  method:  BP
mean: 51.92 std: 0.48 count: 5
dataset:  CIFAR100  method:  CCL
mean: 56.29 std: 0.25 count: 5
dataset:  STL10_cls  method:  BP
mean: 51.27 std: 1.90 count: 5
dataset:  STL10_cls  method:  CCL
mean: 45.28 std: 2.58 count: 5


/var/folders/q7/s2kfc0s17x901pb1hbky39mh0000gn/T/ipykernel_5706/1683781218.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ds_mean = ds.groupby(["grad_clip_F", "lr_F", "lr_B", "mmt_F", "mmt_B", "wd_F", "wd_B", "wd_B", "bw_bn", "loss_scale_C"]).mean()
/var/folders/q7/s2kfc0s17x901pb1hbky39mh0000gn/T/ipykernel_5706/1683781218.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ds_std = ds.groupby(["grad_clip_F", "lr_F", "lr_B", "mmt_F", "mmt_B", "wd_F", "wd_B", "wd_B", "bw_bn", "loss_scale_C"]).std()
/var/folders/q7/s2kfc0s17x901pb1hbky39mh0000gn/T/ipykernel_5706/1683781218.py:7: FutureWarning: The default

AssertionError: 

In [168]:
ds_mean.head(30)

val_acc  \
grad_clip_F lr_F lr_B mmt_F mmt_B wd_F     wd_B     wd_B     bw_bn loss_scale_C             
0.1         0.3  0.3  0.9   0.9   0.000001 0.000001 0.000001 0     0.5           0.608000   
                                                                   1.0           0.631200   
                                                                   2.0           0.635200   
            0.5  0.5  0.9   0.9   0.000001 0.000001 0.000001 0     0.5           0.614800   
                                                                   1.0           0.624000   
                                                                   2.0           0.624400   
0.3         0.3  0.3  0.9   0.9   0.000001 0.000001 0.000001 0     0.5           0.606500   
                                                                   1.0           0.616250   
                                                                   2.0           0.637000   
            0.5  0.5  0.9   0.9   0.000001 0.000001 0.000001 0     0.5           0.634250   
                                                                   1.0           0.632250   
                                                                   2.0           0.646750   
            1.0  1.0  0.9   0.9   0.000001 0.000001 0.000001 0     0.5           0.622000   
                                                                   1.0           0.640000   
                                                                   2.0           0.646667   
            1.5  1.5  0.9   0.9   0.000001 0.000001 0.000001 0     0.5           0.607333   
                                                                   1.0           0.640667   
                                                                   2.0           0.641000   
0.5         0.3  0.3  0.9   0.9   0.000001 0.000001 0.000001 0     0.5           0.608750   
                                                                   1.0           0.615000   
                                                                   2.0           0.643000   
            0.5  0.5  0.9   0.9   0.000001 0.000001 0.000001 0     0.5           0.627800   
                                                                   1.0           0.630667   
                                                                   2.0           0.647000   
            1.0  1.0  0.9   0.9   0.000001 0.000001 0.000001 0     0.5           0.598500   
                                                                   1.0           0.627500   
                                                                   2.0           0.653500   
            1.5  1.5  0.9   0.9   0.000001 0.000001 0.000001 0     0.5           0.615000   
                                                                   1.0           0.617500   
                                                                   2.0           0.642500   

                                                                                 test_acc  \
grad_clip_F lr_F lr_B mmt_F mmt_B wd_F     wd_B     wd_B     bw_bn loss_scale_C             
0.1         0.3  0.3  0.9   0.9   0.000001 0.000001 0.000001 0     0.5           0.612025   
                                                                   1.0           0.621525   
                                                                   2.0           0.605150   
            0.5  0.5  0.9   0.9   0.000001 0.000001 0.000001 0     0.5           0.613100   
                                                                   1.0           0.579325   
                                                                   2.0           0.577350   
0.3         0.3  0.3  0.9   0.9   0.000001 0.000001 0.000001 0     0.5           0.612328   
                                                                   1.0           0.612031   
                                                                   2.0           0.583937   
            0.5  0.5  0.9   0.9   0.000001 0.000001 0.000001 0     0.5           0

In [169]:
row_with_highest_val_acc

(0.5, 1.0, 1.0, 0.9, 0.9, 1e-06, 1e-06, 1e-06, 0, 2.0)

In [4]:
# MNIST

ds = dataset[dataset["dataset"] == "MNIST"]
del ds["Runtime"]
ds = ds[ds.epochs_backward == 5]
ds = ds[(ds.loss_feedback != "DRL") | ((ds.loss_feedback == "DRL") & (ds.learning_rate == 4) & (ds.learning_rate_backward == 0.0002) & (ds.stepsize == 0.04))]
ds = ds[(ds.loss_feedback != "L-DRL") | ((ds.loss_feedback == "L-DRL") & (ds.learning_rate == 4) & (ds.learning_rate_backward == 0.0002) & (ds.stepsize == 0.04))]
ds = ds[~ds.duplicated(subset=["Name", "loss_feedback", "seed", "learning_rate", "learning_rate_backward", "stepsize"])]
pd.pivot_table(ds, values='test accuracy', index=['algorithm', 'loss_feedback'], columns=['dataset'], aggfunc="mean")

dataset                    MNIST
algorithm loss_feedback         
BP        DTP            0.98065
DTP       DRL            0.93250
          DTP            0.97308
          L-DRL          0.93284
DTP-BN    DTP            0.97110
FWDTP     DTP            0.89836
FWDTP-BN  DTP            0.97204

In [5]:
pd.pivot_table(ds, values='test accuracy', index=['algorithm', 'loss_feedback'], columns=['dataset'], aggfunc="std")

dataset                     MNIST
algorithm loss_feedback          
BP        DTP            0.004036
DTP       DRL            0.000914
          DTP            0.000626
          L-DRL          0.004496
DTP-BN    DTP            0.001490
FWDTP     DTP            0.001146
FWDTP-BN  DTP            0.001424

In [11]:
# MNIST
ds = dataset[dataset["dataset"] == "FashionMNIST"]
del ds["Runtime"]
ds = ds[ds.epochs_backward == 5]
ds = ds[(ds.algorithm != "BP") | ((ds.algorithm == "BP") & (ds.learning_rate == 0.1))]
ds = ds[(ds.loss_feedback != "DRL") | ((ds.loss_feedback == "DRL") & (ds.learning_rate == 4) & (ds.learning_rate_backward == 0.002) & (ds.stepsize == 0.008))]
ds = ds[(ds.loss_feedback != "L-DRL") | ((ds.loss_feedback == "L-DRL") & (ds.learning_rate == 2) & (ds.learning_rate_backward == 0.001) & (ds.stepsize == 0.02))]
ds = ds[~ds.duplicated(subset=["Name", "loss_feedback", "seed", "learning_rate", "learning_rate_backward", "stepsize"])]
pd.pivot_table(ds, values='test accuracy', index=['algorithm', 'loss_feedback'], columns=['dataset'], aggfunc="mean")

dataset                  FashionMNIST
algorithm loss_feedback              
BP        DTP                 0.89238
DTP       DRL                 0.83342
          DTP                 0.88272
          L-DRL               0.83514
FWDTP-BN  DTP                 0.88278

In [12]:
pd.pivot_table(ds, values='test accuracy', index=['algorithm', 'loss_feedback'], columns=['dataset'], aggfunc="std")

dataset                  FashionMNIST
algorithm loss_feedback              
BP        DTP                0.007128
DTP       DRL                0.001785
          DTP                0.003154
          L-DRL              0.002004
FWDTP-BN  DTP                0.002333

In [9]:
ds = dataset[dataset["dataset"] == "CIFAR10"]
del ds["Runtime"]
ds["algorithm"].unique()
ds = ds[ds.epochs_backward == 5]
ds = ds[(ds.algorithm != "BP") | ((ds.algorithm == "BP") & (ds.learning_rate == 0.02))]
# pd.pivot_table(ds[ds.algorithm == "BP"], values='test accuracy', index=['learning_rate'], columns=['dataset'], aggfunc="mean")
ds = ds[(ds.loss_feedback != "DRL") | ((ds.loss_feedback == "DRL") & (ds.learning_rate == 2) & (ds.learning_rate_backward == 0.004) & (ds.stepsize == 0.008))]
ds = ds[(ds.loss_feedback != "L-DRL") | ((ds.loss_feedback == "L-DRL") & (ds.learning_rate == 0.2) & (ds.learning_rate_backward == 0.0004) & (ds.stepsize == 0.04))]
ds = ds[~ds.duplicated(subset=["Name", "loss_feedback", "seed", "learning_rate", "learning_rate_backward", "stepsize"])]
pd.pivot_table(ds, values='test accuracy', index=['algorithm', 'loss_feedback'], columns=['dataset'], aggfunc="mean")

dataset                  CIFAR10
algorithm loss_feedback         
BP        DTP            0.53896
DTP       DRL            0.45598
          DTP            0.48800
          L-DRL          0.45420
DTP-BN    DTP            0.45256
FWDTP     DTP            0.35800
FWDTP-BN  DTP            0.50774

In [76]:
pd.pivot_table(ds, values='test accuracy', index=['algorithm', 'loss_feedback'], columns=['dataset'], aggfunc="std")

dataset                   CIFAR10
algorithm loss_feedback          
BP        DTP            0.011425
DTP       DRL            0.004532
          DTP            0.006700
          L-DRL          0.004201
DTP-BN    DTP            0.005097
FWDTP     DTP            0.001032
FWDTP-BN  DTP            0.000918

In [82]:
ds = dataset[dataset["dataset"] == "CIFAR100"]
del ds["Runtime"]
ds["algorithm"].unique()
ds = ds[ds.epochs_backward == 5]
ds = ds[(ds.algorithm != "BP") | ((ds.algorithm == "BP") & (ds.learning_rate == 0.004))]
pd.pivot_table(ds[ds.algorithm == "BP"], values='test accuracy', index=['learning_rate'], columns=['dataset'], aggfunc="mean")
ds = ds[(ds.loss_feedback != "DRL") | ((ds.loss_feedback == "DRL") & (ds.learning_rate == 0.8) & (ds.learning_rate_backward == 0.002) & (ds.stepsize == 0.02))]
ds = ds[(ds.loss_feedback != "L-DRL") | ((ds.loss_feedback == "L-DRL") & (ds.learning_rate == 2) & (ds.learning_rate_backward == 0.0001) & (ds.stepsize == 0.008))]
ds = ds[~ds.duplicated(subset=["Name", "loss_feedback", "seed", "learning_rate", "learning_rate_backward", "stepsize"])]
pd.pivot_table(ds, values='test accuracy', index=['algorithm', 'loss_feedback'], columns=['dataset'], aggfunc="mean")

dataset                  CIFAR100
algorithm loss_feedback          
BP        DTP            0.237025
DTP       DRL            0.204780
          DTP            0.229960
          L-DRL          0.206260
DTP-BN    DTP            0.181200
FWDTP     DTP            0.153720
FWDTP-BN  DTP            0.230440

In [83]:
pd.pivot_table(ds, values='test accuracy', index=['algorithm', 'loss_feedback'], columns=['dataset'], aggfunc="std")

dataset                  CIFAR100
algorithm loss_feedback          
BP        DTP            0.002213
DTP       DRL            0.001807
          DTP            0.000856
          L-DRL          0.001638
DTP-BN    DTP            0.002082
FWDTP     DTP            0.001192
FWDTP-BN  DTP            0.001869

In [80]:
ds[(ds.algorithm=="BP") & (ds.learning_rate==0.004)]

,Name,dataset,algorithm,loss_feedback,test accuracy,State,learning_rate,learning_rate_backward,train accuracy,seed,activation_function,epochs_backward,stepsize
11,CIFAR100_BP,CIFAR100,BP,DTP,0.2361,finished,0.004,0.001,0.34102,5,tanh,5,0.01
41,CIFAR100_BP,CIFAR100,BP,DTP,0.2400,finished,0.004,0.001,0.34256,2,tanh,5,0.01
168,CIFAR100_BP,CIFAR100,BP,DTP,0.2372,finished,0.004,0.001,0.34252,3,tanh,5,0.01
393,CIFAR100_BP,CIFAR100,BP,DTP,0.2348,finished,0.004,0.001,0.33832,4,tanh,5,0.01


# CCL

In [23]:
# MNIST
PATH = "./wandb_classification_CCL_0502.csv"
dataset = pd.read_csv(PATH)
ds = dataset[dataset["dataset"] == "MNIST"]
# del ds["Runtime"]
ds = ds[ds.grad_clip_B == 1]
ds = ds[ds.grad_clip_F == 1]
ds = ds[ds.lr_B == 2]
ds = ds[ds.lr_F == 2]
ds = ds[ds.epochs == 100]
ds = ds[~ds.duplicated()]
ds = ds.dropna()
print(ds.test_acc.mean(), ds.test_acc.std())

0.9815400000000001 0.0011107554986484476


In [24]:
# MNIST
PATH = "./wandb_classification_CCL_0502.csv"
dataset = pd.read_csv(PATH)
ds = dataset[dataset["dataset"] == "FashionMNIST"]
# del ds["Runtime"]
ds = ds[ds.grad_clip_B == 1]
ds = ds[ds.grad_clip_F == 1]
ds = ds[ds.lr_B == 2]
ds = ds[ds.lr_F == 2]
ds = ds[ds.epochs == 100]
ds = ds[~ds.duplicated()]
ds = ds.dropna()
print(ds.test_acc.mean(), ds.test_acc.std())

0.879975 0.004778149671757301


In [40]:
# MNIST
PATH = "./wandb_classification_CCL_0502.csv"
dataset = pd.read_csv(PATH)
ds = dataset[dataset["dataset"] == "CIFAR10"]
# del ds["Runtime"]
ds = ds[ds.grad_clip_B == 1]
ds = ds[ds.grad_clip_F == 1]
ds = ds[ds.lr_B == 0.5]
ds = ds[ds.lr_F == 0.5]
ds = ds[ds.epochs == 100]
ds = ds[~ds.duplicated(subset=["dataset","act_B","grad_clip_B","seed","lr_B","lr_F"])]
ds = ds.dropna()
print(ds.test_acc.mean(), ds.test_acc.std())

0.5212999999999999 0.0021319005605327707


In [47]:
# MNIST
PATH = "./wandb_classification_CCL_0502.csv"
dataset = pd.read_csv(PATH)
ds = dataset[dataset["dataset"] == "CIFAR100"]
# del ds["Runtime"]
ds = ds[ds.grad_clip_B == 1]
ds = ds[ds.grad_clip_F == 1]
ds = ds[ds.lr_B == 4]
ds = ds[ds.lr_F == 4]
ds = ds[ds.epochs == 100]
ds = ds[~ds.duplicated(subset=["dataset","act_B","grad_clip_B","seed","lr_B","lr_F"])]
ds = ds.dropna()
print(ds.test_acc.mean(), ds.test_acc.std())

0.21653333333333333 0.0014571661996262998
